# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Sun Sep 19 16:22:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 457.49       Driver Version: 457.49       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   46C    P8     1W /  N/A |    370MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2018년 Celltrion Data**

In [5]:
lexicon_2018 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2018_celltrion.csv')

lexicon_2018.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,셀트리온,68270,매일경제,2018010104,[2018 경제기상도] `1월효과` 기대 커진 증시…반도체·바이오株 계속 달릴듯,http://news.mk.co.kr/newsRead.php?year=2018&no...,\n\n\n 지난해 연말 조정장세는 올해 초 코스피와 코스닥시장 '1월 효과'의 기...,2018-01-02,4,206047,209825,204757,208167,1176443,0.021709,1,0,지난해 연말 정장 올해 코스피 코스닥시장 효과 폭제 전망 장사 실적 단기 급상승 우...


In [6]:
lexicon_2018[lexicon_2018['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2018.dropna(axis=0, inplace=True)
lexicon_2018 = lexicon_2018.reset_index(drop=True)
lexicon_2018.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [10]:
pos_lexicon_2018 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2018['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2018['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2018['Tokenization'][x].split())):
                    if lexicon_2018['Tokenization'][x].split()[w] == list(set(lexicon_2018['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2018['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2018['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2018 = pos_lexicon_2018.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2018['date'][x], positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2018['Tokenization'][x].split()))[z])))

***2018-01-02 Cosine Similarity between <강세> & <약세> : 0.7028064727783203
***2018-01-02 Cosine Similarity between <개선> & <개선> : 0.9999999403953552
***2018-01-02 Cosine Similarity between <개선> & <방안> : 0.5635414123535156
***2018-01-02 Cosine Similarity between <실현> & <차익> : 0.531851053237915
***2018-01-02 Cosine Similarity between <우세> & <강세> : 0.5120405554771423
***2018-01-02 Cosine Similarity between <중요한> & <가장> : 0.5371253490447998
***2018-01-02 Cosine Similarity between <증대> & <확대> : 0.5193238258361816
***2018-01-02 Cosine Similarity between <증대> & <저하> : 0.5609553456306458
***2018-01-02 Cosine Similarity between <증대> & <감소> : 0.6517642736434937
***2018-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2018-01-02 Cosine Similarity between <추진> & <추진> : 0.9999999403953552
***2018-01-02 Cosine Similarity between <해결> & <문제> : 0.6186186075210571
***2018-01-02 Cosine Similarity between <해결> & <방안> : 0.5310750603675842
***2018-01-02 Cosine Similarity between <호황> & <업황>

In [11]:
pos_lexicon_2018

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2018-01-02,1,강세,약세,0.702806,2
1,2018-01-02,1,개선,개선,1.000000,1
2,2018-01-02,1,개선,방안,0.563541,3
3,2018-01-02,1,실현,차익,0.531851,1
4,2018-01-02,1,우세,강세,0.512041,2
...,...,...,...,...,...,...
9947,2019-01-02,2237,우세,강세,0.512041,3
9948,2019-01-02,2237,장점,특징,0.644235,1
9949,2019-01-02,2238,강세,약세,0.702806,1
9950,2019-01-02,2238,증대,감소,0.651764,6


In [12]:
pos_lexicon_2018.to_csv('../../../../Code/Data/Test/Stock-Year/pos_celltrion_2018.csv', index=False)

In [13]:
a = list(set(list(pos_lexicon_2018['news_word'])))

a

['실패',
 '상쇄하',
 '엠에스',
 '중인',
 '출렁거릴',
 '애블린',
 '정찬',
 '이뮤노메딕스',
 '차익',
 '코스포켐텍',
 '루브리컨츠',
 '횡보세',
 '계획',
 '시너지',
 '로슈사',
 '이니텍이니텍',
 '침체',
 '콘스프',
 '기승준',
 '낸셜컨설팅',
 '주지영',
 '박유악',
 '맥쿼리인프라',
 '극복',
 '우세',
 '삭사틸린',
 '아달리무맙',
 '사하나',
 '과잉반응',
 '텔레칩스텔레',
 '테믹시스',
 '진행',
 '유병삼',
 '최진명',
 '최선',
 '파머슈티컬',
 '런칭돼',
 '장점',
 '플릭시진',
 '드래콘',
 '인플릭시맵',
 '강력',
 '구현',
 '급락',
 '해법',
 '개선',
 '전개',
 '메디프론메디프론',
 '이노톡스',
 '유데나필',
 '텍오스코텍',
 '삼우엠스',
 '랩스커버리',
 '프럼파스트',
 '셜텍크루셜텍',
 '웃돌',
 '제넥텐',
 '들보',
 '이점',
 '올릭스올릭스',
 '덧붙엿다',
 '특징',
 '확보',
 '판매승인',
 '핑거펀',
 '쎄미켐동진쎄미켐',
 '언택트',
 '의미',
 '단점',
 '특성',
 '루텍재영솔루텍',
 '리레이팅',
 '우호',
 '확대',
 '가장',
 '활기',
 '티플랙스',
 '펀인터렉티브',
 '급증',
 '팽팽하',
 '셀루메드셀루메드',
 '일진일퇴',
 '문제',
 '자공시',
 '벤처케피탈',
 '강세',
 '안트로젠안트로젠',
 '프럼파스트프',
 '윤여삼',
 '약세',
 '나노메딕스',
 '케이에스',
 '캔서롭캔서롭',
 '중요',
 '휴톡스',
 '충족',
 '이더펠리체',
 '본격',
 '코람코',
 '불황',
 '한통프리텔',
 '압디사',
 '방안',
 '앰블린파트너스',
 '웃돈',
 '특별도',
 '솔리암페톨',
 '저하',
 '감소',
 '티플랙스티플랙스',
 '문태룡',
 '삼륭물산삼륭물산',
 '겹악재',
 '확충',
 '기록',
 '롱숏펀드',
 '보통신',

In [14]:
b = list(set(list(pos_lexicon_2018[pos_lexicon_2018['cosine_similarity']>=0.7]['news_word'])))

b

['실패',
 '최선',
 '휴메딕스휴메딕스',
 '추진',
 '단점',
 '밑돈',
 '충족',
 '장점',
 '개선',
 '불황',
 '약세',
 '쎄미켐동진쎄미켐',
 '셀루메드셀루메드']

In [15]:
c = list(set(list(pos_lexicon_2018[pos_lexicon_2018['cosine_similarity']>=0.65]['news_word'])))

c

['실패',
 '충족',
 '개선',
 '불황',
 '콘엘비세미콘',
 '이뮤노메딕스',
 '추진',
 '밑돈',
 '웃돌',
 '감소',
 '삼륭물산삼륭물산',
 '쎄미켐동진쎄미켐',
 '단점',
 '메디콕스메디콕스',
 '중앙디앤엠',
 '셀루메드셀루메드',
 '최선',
 '휴메딕스휴메딕스',
 '증가',
 '장점',
 '것이므',
 '안트로젠안트로젠',
 '적극',
 '약세']